In [ ]:
#|default_exp mic

# Mic streaming and recording

In [ ]:
#| export
import io
import sys 
import queue

import zmq
import fire
import numpy as np
import sounddevice as sd
import speech_recognition as sr
from pydub import AudioSegment
from fastcore.basics import store_attr

In [ ]:
#| export 
from asr_teach.utils import SAMPLE_RATE, DEVICE, BLOCK_DURATION, DTYPE
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

In [ ]:
#| export

class AudioStream:
    def __init__(self,
                 sample_rate: float = None,
                 device: int = 0,
                 dtype: str = DTYPE,
                 addr: str = ADDR,
                 port: int = PORT,
                 protocol: str = PROTOCOL):
        store_attr()
        
        # create the socket
        context = zmq.Context()
        socket = context.socket(zmq.PUSH)
        socket.bind(f'{protocol}://{addr}:{port}')
        self.socket = socket
        
        # queue to hold live mic data
        self.q = queue.Queue()
        
        
    def run(self):
        raise NotImplementedError
        
    def setup_stream(self):
        raise NotImplementedError
        
    def send_audio(self, indata, flags=0, copy=True, track=False):
        '''Sends a numpy array in a multi-part message.
        
        The first part of the message has the shape and type of the array.
        The second message has the raw array bytes. 
        '''
        # send the array info
        md = {'dtype': str(indata.dtype),
              'shape': indata.shape}
        self.socket.send_json(md, zmq.SNDMORE)
        # sends the array data
        self.socket.send(indata, flags, copy=copy, track=track)
        

In [ ]:
#| export

class SoundDeviceMic(AudioStream):
    def __init__(self, *args,
                 block_duration: int = 2500,
                 **kwargs):
        super().__init__(*args, **kwargs)
        store_attr()
        self.setup_stream()
        
    def setup_stream(self):
        
        # setting the sample rate and mic buffer size
        self.sample_rate = self.sample_rate or sd.query_devices(self.device, 'input')['default_samplerate']
        self.num_samples = int(self.sample_rate * self.block_duration)
        self.blocksize = self.num_samples // 1000

        def enqueue_audio(indata, frames, time, status):
            '''Places audio data on the queue.
            '''
            if any(indata):
                self.q.put(indata)
        
        # create the microphone streaming object
        self.stream = sd.InputStream(device=self.device,
                                     channels=1,
                                     callback=enqueue_audio,
                                     dtype=self.dtype,
                                     blocksize=self.blocksize,
                                     samplerate=self.sample_rate)
        
        
    def run(self):
        '''Streams audio until the user stops or interrupts the process.
        '''
        # start the microphone stream
        self.stream.start()
        print('Streaming live mic audio...')
        while True:
            try:
                # send any audio on the queue
                if not self.q.empty():
                    self.send_audio(self.q.get())
                    
            except KeyboardInterrupt:
                print('User keyboard interrupt, stopping mic...')
                self.stream.close()
                break
                
            except Exception as e:
                print(f'Exception: {e}')
                raise
                
        print('Live mic stopped.')
            

## Microphone streaming via the SpeechRecognition repo

In [ ]:
#| export

class SpeechRecogMic(AudioStream):
    def __init__(self, *args,
                 energy: int = 500,
                 pause: float = 0.8,
                 dynamic_energy: bool = False,
                 **kwargs):
        super().__init__(*args, **kwargs)
        store_attr()
        self.setup_stream()
        
    def setup_stream(self):
        
        # load the speech recognizer with CLI settings
        recog = sr.Recognizer()
        recog.energy_threshold = self.energy
        recog.pause_threshold = self.pause
        recog.dynamic_energy_threshold = self.dynamic_energy
        self.recog = recog
        
        self.mic = sr.Microphone(device_index=self.device,
                                 sample_rate=self.sample_rate)

    def run(self):
        with self.mic as source:
            print("Starting mic stream...")
            while True:
                # record audio stream into wav
                try:
                    data = self.recog.listen(source)
                    audio = np.frombuffer(data.frame_data, dtype=np.int16)
                    audio = audio.flatten().astype(np.float32) / 32768.0
                    self.send_audio(audio)
                    
                except KeyboardInterrupt:
                    print('User keyboard interrupt, stopping mic...')
                    break

                except Exception as e:
                    print(f'Exception: {e}')
                    raise
            print('Mic stopped.')

In [ ]:
#| export

def stream(sample_rate: float = SAMPLE_RATE,
           device: int = DEVICE,
           block_duration: int = BLOCK_DURATION,
           **kwargs):
    '''Entrypoint for live microphone streaming over a port.
    '''
    # create and start the microphone stream
    mic = MicStream(sample_rate, device, block_duration, **kwargs)
    mic.run()